In [ ]:
from pyspark.sql import SparkSession

# Tworzę sesję Sparka
spark = SparkSession.builder \
    .appName("AnalizaOpenAQ") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

print("Sesja Sparka uruchomiona!")

26/01/09 20:05:39 WARN util.Utils: Your hostname, node1 resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
26/01/09 20:05:39 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/09 20:05:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Sesja Sparka uruchomiona!


In [ ]:
df = spark.table("air_quality")

df.limit(10).toPandas()

26/01/09 20:06:23 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


,location_id,location_name,latitude,longitude,parameter,datetimelocal,value,date
0,28,Ursynów,52.160773,21.039167,no2,30-01-2016 00:00:00,17.1,2026-01-05
1,28,Ursynów,52.160773,21.039167,no2,31-01-2016 00:00:00,10.7,2026-01-05
2,28,Ursynów,52.160773,21.039167,no2,01-02-2016 00:00:00,19.4,2026-01-05
3,28,Ursynów,52.160773,21.039167,no2,02-02-2016 00:00:00,16.5,2026-01-05
4,28,Ursynów,52.160773,21.039167,no2,03-02-2016 00:00:00,11.3,2026-01-05
5,28,Ursynów,52.160773,21.039167,no2,04-02-2016 00:00:00,17.7,2026-01-05
6,28,Ursynów,52.160773,21.039167,no2,05-02-2016 00:00:00,21.0,2026-01-05
7,28,Ursynów,52.160773,21.039167,no2,06-02-2016 00:00:00,23.3,2026-01-05
8,28,Ursynów,52.160773,21.039167,no2,07-02-2016 00:00:00,13.0,2026-01-05
9,28,Ursynów,52.160773,21.039167,no2,08-02-2016 00:00:00,18.0,2026-01-05


In [ ]:
from pyspark.sql.functions import col, to_timestamp

df_unique = df.dropDuplicates()

df_parsed = df_unique.withColumn(
    "datetimelocal",
    to_timestamp(col("datetimelocal"), "dd-MM-yyyy HH:mm:ss")
)

df_filtered = df_parsed.filter(col("datetimelocal") >= "2025-12-01")

print(f"Liczba wierszy przed: {df.count()}")
print(f"Liczba wierszy po filtracji: {df_filtered.count()}")

df_filtered.limit(10).toPandas()

Liczba wierszy przed: 47673


Liczba wierszy po filtracji: 1007


,location_id,location_name,latitude,longitude,parameter,datetimelocal,value,date
0,2872756,Biala,52.238607,20.990778,pm1,2025-12-23,10.20,2026-01-05
1,2876975,Sadul,52.207899,21.148612,pm25,2025-12-25,51.40,2026-01-05
2,2876975,Sadul,52.207899,21.148612,temperature,2025-12-24,2.65,2026-01-05
3,3270912,Sowińskiego 25,52.224002,20.935344,temperature,2025-12-28,6.81,2026-01-05
4,5075698,Warsaw (Hala Mirowska),52.238825,20.998093,temperature,2025-12-30,3.10,2026-01-05
5,6040748,"Planety 23, Jozefoslaw",52.098158,21.040892,relativehumidity,2025-12-15,54.20,2026-01-05
6,2872756,Biala,52.238607,20.990778,pm25,2025-12-09,27.00,2026-01-05
7,3270912,Sowińskiego 25,52.224002,20.935344,pm1,2025-12-03,19.30,2026-01-05
8,3270912,Sowińskiego 25,52.224002,20.935344,pm1,2025-12-06,20.50,2026-01-05
9,3270912,Sowińskiego 25,52.224002,20.935344,relativehumidity,2025-12-06,64.00,2026-01-05


In [4]:
df_filtered.printSchema()

root
 |-- location_id: long (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- parameter: string (nullable = true)
 |-- datetimelocal: timestamp (nullable = true)
 |-- value: double (nullable = true)
 |-- date: string (nullable = true)



In [ ]:
df_filtered.toPandas()


,location_id,location_name,latitude,longitude,parameter,datetimelocal,value,date
0,2872756,Biala,52.238607,20.990778,pm1,2025-12-23,10.20,2026-01-05
1,2876975,Sadul,52.207899,21.148612,pm25,2025-12-25,51.40,2026-01-05
2,2876975,Sadul,52.207899,21.148612,temperature,2025-12-24,2.65,2026-01-05
3,3270912,Sowińskiego 25,52.224002,20.935344,temperature,2025-12-28,6.81,2026-01-05
4,5075698,Warsaw (Hala Mirowska),52.238825,20.998093,temperature,2025-12-30,3.10,2026-01-05
...,...,...,...,...,...,...,...,...
1002,3180107,Belgradzka,52.143305,21.066756,temperature,2025-12-20,9.49,2026-01-05
1003,3270912,Sowińskiego 25,52.224002,20.935344,um003,2025-12-28,433.00,2026-01-05
1004,5075698,Warsaw (Hala Mirowska),52.238825,20.998093,relativehumidity,2026-01-01,57.10,2026-01-05
1005,6040748,"Planety 23, Jozefoslaw",52.098158,21.040892,pm1,2025-12-10,23.50,2026-01-05


In [ ]:
from pyspark.sql.functions import avg, desc, round

analiza_stacji = df_filtered.groupBy("location_name", "parameter") \
    .agg(round(avg("value"), 2).alias("average_value")) \
    .orderBy(desc("average_value"))

analiza_stacji.toPandas()

,location_name,parameter,average_value
0,Belgradzka,um003,3793.50
1,Sowińskiego 25,um003,3152.03
2,Sadul,um003,2992.86
3,Biala,um003,2748.23
4,"Planety 23, Jozefoslaw",um003,1029.23
5,Warsaw (Hala Mirowska),um003,889.57
6,Belgradzka,relativehumidity,60.76
7,Sowińskiego 25,relativehumidity,60.53
8,Biala,relativehumidity,58.51
9,Warsaw (Hala Mirowska),relativehumidity,58.14


In [8]:
# Pobieram pełne dane (juz przefiltrowane datami) do lokalnego Pandasa
pdf = df_filtered.toPandas()
pdf = pdf.sort_values(by="datetimelocal")
print("Dane gotowe do rysowania!")

Dane gotowe do rysowania!


In [ ]:
import plotly.graph_objects as go
import plotly.express as px


dostepne_parametry = pdf['parameter'].unique()
lokalizacje = pdf['location_name'].unique()

fig = go.Figure()

for param in dostepne_parametry:
    df_subset = pdf[pdf['parameter'] == param]
    
    for loc in lokalizacje:
        df_loc = df_subset[df_subset['location_name'] == loc]
        
        if df_loc.empty:
            continue
            
        fig.add_trace(go.Scatter(
            x=df_loc['datetimelocal'],
            y=df_loc['value'],
            name=loc, 
            mode='lines',
            visible=(param == dostepne_parametry[0]),
            customdata=[param] * len(df_loc)
        ))

przyciski = []

for param in dostepne_parametry:
    visibility = []
    for trace in fig.data:
        if trace.customdata[0] == param:
            visibility.append(True)
        else:
            visibility.append(False)
            
    button = dict(
        label=param,
        method="update",
        args=[{"visible": visibility}, 
              {"title": f"Parametr: {param}",
               "yaxis": {"title": f"Wartość ({param})"}}], 
    )
    przyciski.append(button)

fig.update_layout(
    title=f"Parametr: {dostepne_parametry[0]}",
    template="plotly_white",
    xaxis=dict(
        rangeslider=dict(visible=False), 
        type="date"
    ),
    updatemenus=[{
        "buttons": przyciski,
        "direction": "up",     
        "showactive": True,
        "x": 1.0,               
        "y": -0.2,              
        "xanchor": "right",
        "yanchor": "top"
    }],
    margin=dict(b=100)
)

fig.show()

In [24]:
mapa_dane = pdf.groupby(['location_name', 'latitude', 'longitude', 'parameter'])['value'].mean().reset_index()
dostepne_parametry = mapa_dane['parameter'].unique()

fig = go.Figure()

for param in dostepne_parametry:
    df_p = mapa_dane[mapa_dane['parameter'] == param]
    
    fig.add_trace(go.Scattermapbox(
        lat=df_p['latitude'],
        lon=df_p['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,               
            color=df_p['value'],
            
            colorscale='RdYlGn_r',
            
            showscale=True,
            colorbar=dict(title=f"Wartość ({param})")
        ),
        text=df_p['location_name'], 
        hovertemplate=f"<b>%{{text}}</b><br>" +
                      f"Parametr: {param}<br>" +
                      f"Średnia wartość: %{{marker.color:.2f}}<extra></extra>",
        name=param,
        visible=(param == dostepne_parametry[0])
    ))

przyciski = []
for i, param in enumerate(dostepne_parametry):
    visibility = [False] * len(dostepne_parametry)
    visibility[i] = True
    button = dict(label=param, method="update",
                  args=[{"visible": visibility}, {"title": f"Mapa: {param}"}])
    przyciski.append(button)

fig.update_layout(
    mapbox_style="open-street-map",
    title=f"Mapa: {dostepne_parametry[0]}",
    margin={"r":0,"t":50,"l":0,"b":0},
    mapbox=dict(center=dict(lat=52.23, lon=21.01), zoom=10),
    updatemenus=[{
        "buttons": przyciski,
        "direction": "down",
        "x": 0.01, "y": 0.99,
        "bgcolor": "white",
        "showactive": True
    }]
)

fig.show()

/tmp/ipykernel_32378/1193448326.py:9: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [ ]:
df_pivot = pdf.pivot_table(index='datetimelocal', columns='parameter', values='value', aggfunc='mean')

matrix_corr = df_pivot.corr()

fig = px.imshow(matrix_corr,
                text_auto=".2f",                
                aspect="auto",
                title="Co wpływa na co?",
                color_continuous_scale='RdBu_r', 
                origin='lower',
                zmin=-1,                        
                zmax=1)                         

fig.show()

In [23]:
# Wybieram tylko PM2.5 (bo to główny wskaźnik smogu)
df_box = pdf[pdf['parameter'] == 'pm25']

fig = px.box(df_box, 
             x="location_name", 
             y="value", 
             color="location_name",
             title="Kto ma najbardziej stabilne powietrze? (Rozkład PM2.5)",
             points="all",
             hover_data=["datetimelocal"])

fig.update_layout(
    xaxis_title="Stacja",
    yaxis_title="Stężenie PM2.5",
    showlegend=False 
)

fig.show()